# Distracted Driving Detection Model Showcase

In [1]:
# this is where we showcase our final model on a small sample of the data (once we are done creating our best model)

In [ ]:
"""
What to do:
1) load the best model from a pickle file
2) create an image array from a small (less than 5 MB) folder of select test images
3) run the model and showcase the performance
4) make sure the entire notebook takes less than a minute to run
"""

In [1]:
import pickle
import numpy as np
import os
import cv2
import torch

import sys
sys.path.append('./src/')

%load_ext autoreload
%autoreload 1
%aimport main
%aimport Helper

# Load data + models

In [ ]:
#note: models and sample_data aren't created yet

with open('torch_model.pkl') as fid:
    torch_model = pickle.load(fid)
with open('kristens_model.pkl') as fid:
    kristens_model = pickle.load(fid)
with open('illorums_model.pkl') as fid:
    illorums_model = pickle.load(fid)

Xtest, Ytest = [], []
for filename in os.listdir('./sample_data/'):
    img = cv2.imread(path, cv2.IMREAD_COLOR) # read image as numpy array
    img = cv2.resize(img, (64,64)) # resize image so its smaller
    Xtest.append(img)

Xtest = np.asarray(Xtest)
Ytest = np.asarray(Ytest)

# Model 1: PyTorch

In [2]:
if torch.cuda.is_available():
    DTYPE = torch.cuda.FloatTensor #GPU type
else:
    DTYPE = torch.FloatTensor #CPU type

In [ ]:
%autoreload

Xtest_torch = np.transpose(Xtest, (0, 3, 1, 2)) # format dimensions for the model
Xtest_torch = torch.from_numpy(Xtest_torch) # convert to PyTorch tensor
Helper.evaluate(torch_model, Xtest_torch, Ytest_torch, Xtest_torch.shape[0], dtype = DTYPE)

# Model 2:

In [ ]:
%autoreload

# convert to appropriate tensor if needed

Helper.evaluate(torch_model, )

# Model 3:

In [ ]:
%autoreload

# convert to appropriate tensor if needed

Helper.evaluate(torch_model, )

# Final Analysis